In [3]:
import os
from tqdm import tqdm
os.chdir('G:/sources/AI_based_SE/AISE_fooling')

from model import Model
import numpy as np
import torch
from torchvision.datasets import mnist
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [5]:
def main():
    batch_size = 256
    epoch = 100
    
    train_dataset = mnist.MNIST(root='./train', train=True, transform=ToTensor())
    test_dataset = mnist.MNIST(root='./test', train=False, transform=ToTensor())
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    model = Model()
    sgd = SGD(model.parameters(), lr=1e-1)
    cost = CrossEntropyLoss()
    
    previous_correct = 0

    print(len(train_loader))

    for _epoch in range(epoch):
        for idx, (train_x, train_label) in enumerate(train_loader):
            label_np = np.zeros((train_label.shape[0], 10))
            sgd.zero_grad()
            predict_y = model(train_x.float())
            loss = cost(predict_y, train_label.long())
            #if idx % 50 == 0:
            #    print('idx: {}, loss: {}'.format(idx, loss.sum().item()))
            loss.backward()
            sgd.step()

        correct = 0
        _sum = 0

        for idx, (test_x, test_label) in enumerate(test_loader):
            predict_y = model(test_x.float()).detach()
            predict_ys = np.argmax(predict_y, axis=-1)
            label_np = test_label.numpy()
            _ = predict_ys == test_label
            correct += np.sum(_.numpy(), axis=-1)
            _sum += _.shape[0]

        if previous_correct < correct: 
            print('epoch : {},\taccuracy: {:.2f}%'.format(_epoch ,correct / _sum * 100))
            torch.save(model, 'models/mnist_{:.4f}.pkl'.format(correct / _sum))
            previous_correct = correct

if __name__ == '__main__':
    main()

235
epoch : 0,	accuracy: 67.41%
epoch : 1,	accuracy: 90.80%
epoch : 4,	accuracy: 95.53%
epoch : 5,	accuracy: 96.35%
epoch : 6,	accuracy: 97.12%
epoch : 7,	accuracy: 97.45%
epoch : 8,	accuracy: 97.71%
epoch : 9,	accuracy: 97.90%
epoch : 10,	accuracy: 98.00%
epoch : 11,	accuracy: 98.05%
epoch : 12,	accuracy: 98.10%
epoch : 14,	accuracy: 98.14%
epoch : 19,	accuracy: 98.19%
epoch : 27,	accuracy: 98.33%
epoch : 30,	accuracy: 98.44%
epoch : 37,	accuracy: 98.45%
epoch : 39,	accuracy: 98.52%
epoch : 42,	accuracy: 98.68%
epoch : 43,	accuracy: 98.71%
epoch : 48,	accuracy: 98.77%
epoch : 49,	accuracy: 98.79%
epoch : 50,	accuracy: 98.84%
epoch : 59,	accuracy: 98.87%
epoch : 60,	accuracy: 98.89%
epoch : 64,	accuracy: 98.90%
epoch : 77,	accuracy: 98.91%
epoch : 87,	accuracy: 98.93%
